# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [56]:
import pandas as pd

In [57]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [58]:
df1 = df1.drop_duplicates()

In [59]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [60]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [61]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [62]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [63]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [64]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [65]:
from datetime import timedelta, datetime

In [66]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], format='%d/%m/%Y')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

In [67]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [68]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [69]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [70]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [71]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [72]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,33,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,13,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,20,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,12,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,7,The Old Oak,2023-11-16,GBR,Lucky Red Distrib.,14325.79,2417,677270.69,104359,2023-11-28,12,2023-11-27,Monday,5.93
128,8,Casanova Operapop - Il Film,2023-11-27,ITA,Officine Ubu,12261.40,1253,13770.90,1470,2023-11-28,,2023-11-27,Monday,9.79
129,9,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,12158.67,1698,3202640.84,433917,2023-11-28,20,2023-11-27,Monday,7.16
130,10,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,11883.89,1629,651919.52,85325,2023-11-28,12,2023-11-27,Monday,7.30


---

## Export as .csv

In [73]:
import os

In [74]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [75]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [76]:
import seaborn as sns
import matplotlib.pyplot as plt

In [77]:
pd.set_option('display.max_rows', 8)

In [78]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 0 to 130
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            120 non-null    int32         
 1   title                 120 non-null    object        
 2   first_screening_date  120 non-null    object        
 3   nation                120 non-null    object        
 4   distribution          120 non-null    object        
 5   daily_takings         120 non-null    float64       
 6   daily_attendance      120 non-null    int32         
 7   total_takings         120 non-null    float64       
 8   total_attendance      120 non-null    int32         
 9   date_pulled           120 non-null    object        
 10  screening_days        120 non-null    object        
 11  date                  120 non-null    datetime64[ns]
 12  day_of_week           120 non-null    object        
 13  avg_ticket_price      120

In [79]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,date,avg_ticket_price
count,120.00,120.00,120.00,120.00,120.00,120,120.00
mean,5.50,180250.89,25314.53,3706697.48,530866.86,2023-11-21 12:00:00,6.88
min,1.00,8904.68,1253.00,11652.78,1470.00,2023-11-16 00:00:00,4.22
25%,3.00,20538.25,4275.50,418535.15,58848.75,2023-11-18 18:00:00,6.30
50%,5.50,48106.60,7251.00,2074384.39,300455.50,2023-11-21 12:00:00,6.81
75%,8.00,206430.22,28521.25,3365981.69,495806.00,2023-11-24 06:00:00,7.40
max,10.00,1544231.00,211764.00,24253652.01,3524662.00,2023-11-27 00:00:00,10.94
std,2.88,289109.80,40078.48,5761172.24,835445.47,NaN,1.08


In [80]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    0
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [81]:
df1.nunique()

daily_rank               10
title                    19
first_screening_date     12
nation                    5
                       ... 
screening_days           12
date                     12
day_of_week               7
avg_ticket_price        120
Length: 14, dtype: int64

In [82]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,avg_ticket_price
daily_rank,1.00,-0.66,-0.66,-0.47,-0.46,-0.24
daily_takings,-0.66,1.00,0.99,0.66,0.66,0.19
daily_attendance,-0.66,0.99,1.00,0.71,0.71,0.14
total_takings,-0.47,0.66,0.71,1.00,1.00,-0.10
total_attendance,-0.46,0.66,0.71,1.00,1.00,-0.13
avg_ticket_price,-0.24,0.19,0.14,-0.10,-0.13,1.00


In [83]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,avg_ticket_price
title,,,,,,
C'E' Ancora Domani,1.17,777733.21,113989.25,20078783.80,2906224.00,6.68
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,7.20
...,...,...,...,...,...,...
Mary E Lo Spirito Di Mezzanotte,9.50,38548.18,5860.50,68220.25,10738.50,6.58
La Chimera,10.00,12522.45,1942.50,19722.90,3210.50,6.38
Picasso. Un Ribelle A Parigi. Storia Di Una Vita E Di Un Museo,5.00,20611.08,2566.00,20611.08,2566.00,8.03
Casanova Operapop - Il Film,8.00,12261.40,1253.00,13770.90,1470.00,9.79


In [84]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,avg_ticket_price
title,,,,,,
C'E' Ancora Domani,1.17,777733.21,113989.25,20078783.80,2906224.00,6.68
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,7.20
...,...,...,...,...,...,...
Mary E Lo Spirito Di Mezzanotte,9.50,38548.18,5860.50,68220.25,10738.50,6.58
La Chimera,10.00,12522.45,1942.50,19722.90,3210.50,6.38
Picasso. Un Ribelle A Parigi. Storia Di Una Vita E Di Un Museo,5.00,20611.08,2566.00,20611.08,2566.00,8.03
Casanova Operapop - Il Film,8.00,12261.40,1253.00,13770.90,1470.00,9.79
